# Transformer 模型

Transformer 模型是一种在自然语言处理（NLP）任务中广泛使用的深度学习模型，特别是在机器翻译、文本摘要、问答系统等任务中表现出色。Transformer 模型由 Vaswani 等人在 2017 年提出，其核心思想是使用自注意力机制（Self-Attention）和多头注意力机制（Multi-Head Attention）来捕捉输入序列中的依赖关系，而不依赖于传统的循环神经网络（RNN）或卷积神经网络（CNN）。

## 1. 背景

在传统的序列到序列（Seq2Seq）模型中，如循环神经网络（RNN）、长短期记忆网络（LSTM）和门控循环单元（GRU），序列的顺序信息是通过时间步（Time Step）隐式地捕捉的。然而，这些模型在处理长序列时存在梯度消失和计算效率低下的问题。为了解决这些问题，Transformer 模型被提出，完全基于自注意力机制和多头注意力机制，不依赖于时间步。

## 2. Transformer 的核心思想

Transformer 模型的核心思想是通过自注意力机制和多头注意力机制来捕捉输入序列中的依赖关系。具体来说，Transformer 模型由编码器（Encoder）和解码器（Decoder）两部分组成，每部分由多个相同的层堆叠而成。每个层包含两个主要组件：多头注意力机制和前馈神经网络（Feedforward Neural Network）。

## 3. 工作原理

### 3.1 编码器（Encoder）

编码器由多个相同的层堆叠而成，每个层包含以下两个组件：

- **多头自注意力机制（Multi-Head Self-Attention）**：每个注意力头关注输入序列的不同部分，捕捉序列内部的依赖关系。
- **前馈神经网络（Feedforward Neural Network）**：对每个位置的表示进行非线性变换。

每个层的输出通过残差连接（Residual Connection）和层归一化（Layer Normalization）进行处理，以提高模型的稳定性和训练效率。

### 3.2 解码器（Decoder）

解码器也由多个相同的层堆叠而成，每个层包含以下三个组件：

- **多头自注意力机制（Multi-Head Self-Attention）**：与编码器类似，捕捉解码器输入序列内部的依赖关系。
- **多头注意力机制（Multi-Head Attention）**：关注编码器的输出，捕捉编码器和解码器之间的依赖关系。
- **前馈神经网络（Feedforward Neural Network）**：对每个位置的表示进行非线性变换。

每个层的输出同样通过残差连接和层归一化进行处理。

### 3.3 位置编码（Positional Encoding）

由于 Transformer 模型不依赖于时间步，因此需要显式地引入位置信息。位置编码通过将位置向量与输入序列的嵌入向量相加，得到包含位置信息的嵌入向量。

- **公式**：
  \[
  \text{embedding\_with\_position} = \text{embedding} + \text{positional\_encoding}
  \]

![Transformer](https://zh-v2.d2l.ai/_images/transformer.svg "Transformer")

## 4. 优点与局限性

### 4.1 优点

- **并行计算**：Transformer 模型完全基于自注意力机制和多头注意力机制，不依赖于时间步，因此可以并行计算，显著提高计算效率。
- **捕捉长距离依赖**：自注意力机制和多头注意力机制可以捕捉输入序列中的长距离依赖关系，提高模型的性能。
- **灵活性**：Transformer 模型可以处理不同长度的输入序列，适用于多种 NLP 任务。

### 4.2 局限性

- **计算复杂度**：Transformer 模型的计算复杂度较高，尤其是在处理长序列时。
- **可解释性**：虽然 Transformer 模型提高了模型的性能，但它也使得模型的可解释性降低，因为注意力权重是动态计算的，难以直观理解。

## 5. 应用场景

- **机器翻译**：Transformer 模型在机器翻译任务中表现出色，特别是在处理长序列时。
- **文本摘要**：Transformer 模型可以生成简洁且信息量大的文本摘要。
- **问答系统**：Transformer 模型可以帮助生成准确的回答，捕捉问题中的关键部分和相关上下文。

## 6. 总结

Transformer 模型是一种在自然语言处理任务中广泛使用的深度学习模型，通过自注意力机制和多头注意力机制捕捉输入序列中的依赖关系，显著提高了模型的性能。尽管计算复杂度较高，但 Transformer 模型在许多 NLP 任务中表现出色，成为现代深度学习模型的核心组件之一。

## 简单代码实现

### 基于位置的前馈网络

In [ ]:
#@save
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

### 残差连接和层规范化

In [ ]:
#@save
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

### 编码器

In [ ]:
#@save
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [ ]:
#@save
class TransformerEncoder(d2l.Encoder):
    """Transformer编码器"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[
                i] = blk.attention.attention.attention_weights
        return X

### 解码器


In [ ]:
class DecoderBlock(nn.Module):
    """解码器中第i个块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i
        self.attention1 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.attention2 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens,
                                   num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)

    def forward(self, X, state):
        enc_outputs, enc_valid_lens = state[0], state[1]
        # 训练阶段，输出序列的所有词元都在同一时间处理，
        # 因此state[2][self.i]初始化为None。
        # 预测阶段，输出序列是通过词元一个接着一个解码的，
        # 因此state[2][self.i]包含着直到当前时间步第i个块解码的输出表示
        if state[2][self.i] is None:
            key_values = X
        else:
            key_values = torch.cat((state[2][self.i], X), axis=1)
        state[2][self.i] = key_values
        if self.training:
            batch_size, num_steps, _ = X.shape
            # dec_valid_lens的开头:(batch_size,num_steps),
            # 其中每一行是[1,2,...,num_steps]
            dec_valid_lens = torch.arange(
                1, num_steps + 1, device=X.device).repeat(batch_size, 1)
        else:
            dec_valid_lens = None

        # 自注意力
        X2 = self.attention1(X, key_values, key_values, dec_valid_lens)
        Y = self.addnorm1(X, X2)
        # 编码器－解码器注意力。
        # enc_outputs的开头:(batch_size,num_steps,num_hiddens)
        Y2 = self.attention2(Y, enc_outputs, enc_outputs, enc_valid_lens)
        Z = self.addnorm2(Y, Y2)
        return self.addnorm3(Z, self.ffn(Z)), state

In [ ]:
class TransformerDecoder(d2l.AttentionDecoder):
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                DecoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, i))
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        return [enc_outputs, enc_valid_lens, [None] * self.num_layers]

    def forward(self, X, state):
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self._attention_weights = [[None] * len(self.blks) for _ in range (2)]
        for i, blk in enumerate(self.blks):
            X, state = blk(X, state)
            # 解码器自注意力权重
            self._attention_weights[0][
                i] = blk.attention1.attention.attention_weights
            # “编码器－解码器”自注意力权重
            self._attention_weights[1][
                i] = blk.attention2.attention.attention_weights
        return self.dense(X), state

    @property
    def attention_weights(self):
        return self._attention_weights